### King County House Prices Exercise
- This notebook explores and analyzes house prices data in King County, WA. 

In [63]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

### Load Data
- This step loads and data and print a few rows to get a quick look at the data. 

In [8]:
def load_data(file_path):
    '''
    load data, examine shape of the data, check for duplicates, and print out 5 top rows
    Input: a string that indicates where the data is stored
    Output: a dataframe with house prices in King County
    '''
    house_df = pd.read_csv(file_path)
    print(house_df.head())
    print(f"Data includes the following columns and of these types: \n{house_df.dtypes}")
    print(f"Shape of data is: {house_df.shape}")
    
    house_df_dedup = house_df.drop_duplicates()
    print(f"Shape of data after dropping duplicates is: {house_df_dedup.shape}")
    
    return house_df_dedup

In [9]:
house_price = load_data('house_prices.csv')

           id             date     price  bedrooms  bathrooms  sqft_living  \
0  7129300520  20141013T000000  221900.0         3       1.00         1180   
1  6414100192  20141209T000000  538000.0         3       2.25         2570   
2  5631500400  20150225T000000  180000.0         2       1.00          770   
3  2487200875  20141209T000000  604000.0         4       3.00         1960   
4  1954400510  20150218T000000  510000.0         3       2.00         1680   

   sqft_lot  floors  waterfront  view  ...  grade  sqft_above  sqft_basement  \
0      5650     1.0           0     0  ...      7        1180              0   
1      7242     2.0           0     0  ...      7        2170            400   
2     10000     1.0           0     0  ...      6         770              0   
3      5000     1.0           0     0  ...      7        1050            910   
4      8080     1.0           0     0  ...      8        1680              0   

   yr_built  yr_renovated  zipcode      lat     lo

### Alternate Data Types
- After checking the data types, we can see some data types are mismatched. For example, `date` should be of datetime type instead of object type, `waterfront` and `view` should be of category type instead of integer. Let's alter the data types for these columns in this step. 

In [14]:
def alter_type(house_df_dedup):
    '''
    alternate data types if they are inappropriate
    Input: a dataframe with house prices, after dropping duplicates
    Output: a dataframe with appropriate data types for each variables
    '''
    house_df_alt = house_df_dedup.copy()
    
    house_df_alt['date'] = pd.to_datetime(house_df_dedup['date'], errors='coerce')
    print(house_df_alt['date'].head())
    
    house_df_alt['waterfront'] = house_df_dedup['waterfront'].astype('category')
    house_df_alt['view'] = house_df_dedup['view'].astype('category')
    house_df_alt['zipcode'] = house_df_dedup['zipcode'].astype('category')
    print(house_df_alt[['waterfront', 'view','zipcode']].head())
    
    print(house_df_alt.dtypes)
    
    return house_df_alt

In [15]:
house_price_alt = alter_type(house_price)

0   2014-10-13
1   2014-12-09
2   2015-02-25
3   2014-12-09
4   2015-02-18
Name: date, dtype: datetime64[ns]
  waterfront view zipcode
0          0    0   98178
1          0    0   98125
2          0    0   98028
3          0    0   98136
4          0    0   98074
id                        int64
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront             category
view                   category
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                category
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object


### Explore Data
- This step explores the data further, checking for missing values, summary stats, correlations, plots, etc. 

In [50]:
def describe_data(house_df_alt):
    '''
    describes data, includes generating summary stats, checking for missing values, plotting simple histograms, etc. 
    Input: a dataframe of house prices
    Output: print out including summary stats, count of missing values, simple plots, etc. 
            and a dataframe that includes an additional variable: renovated, which is a dummy variable that equals 1
            if a house was ever renovated.
    '''
    print(f"Summary stats for house price data is: \n{house_df_alt.describe()}\n")
    
    for col in house_df_alt.columns:
        print(f"There are {house_df_alt[col].isna().sum()} missing values in {col}.")
    
    house_df_alt['renovated'] = np.where(house_df_alt['yr_renovated']>0, 1, 0)
    
    for col in ['renovated','waterfront','view','zipcode']:
        print(house_df_alt[col].value_counts().sort_values(ascending=False).head(10))
    
    house_df_alt['year'] = house_df_alt['date'].dt.year
    print(house_df_alt['year'].value_counts().sort_values(ascending=False).head())
    
    print(house_df_alt[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 
                        'sqft_above', 'sqft_basement', 'yr_built']].corr().reset_index())
    return house_df_alt

In [51]:
house_price_alt = describe_data(house_price_alt)

Summary stats for house price data is: 
                 id         price      bedrooms     bathrooms   sqft_living  \
count  2.161300e+04  2.161300e+04  21613.000000  21613.000000  21613.000000   
mean   4.580302e+09  5.400881e+05      3.370842      2.114757   2079.899736   
std    2.876566e+09  3.671272e+05      0.930062      0.770163    918.440897   
min    1.000102e+06  7.500000e+04      0.000000      0.000000    290.000000   
25%    2.123049e+09  3.219500e+05      3.000000      1.750000   1427.000000   
50%    3.904930e+09  4.500000e+05      3.000000      2.250000   1910.000000   
75%    7.308900e+09  6.450000e+05      4.000000      2.500000   2550.000000   
max    9.900000e+09  7.700000e+06     33.000000      8.000000  13540.000000   

           sqft_lot        floors     condition         grade    sqft_above  \
count  2.161300e+04  21613.000000  21613.000000  21613.000000  21613.000000   
mean   1.510697e+04      1.494309      3.409430      7.656873   1788.390691   
std    4.14

### Regression Analysis
- This step analyzes the data in a regression analysis using OLS with fixed effects and cluster standard errors, if applicable. 

In [39]:
print(house_price_alt.columns)

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'renovated', 'year'],
      dtype='object')


In [58]:
def reg_model(house_df_alt, y, cluster_group):
    '''
    run a regression model to regress prices (or other outcome metrics) on other variables 
        to quantify their correlations with prices.
    Input: 1. a dataframe with house prices and other control variables, 
           all variables should be of the appropriate data types;
           2. an outcome variable.
    Output: print out of regression analysis summary.
    '''
    reg_formula = f'{y} ~ bedrooms + bathrooms + sqft_living + sqft_lot + floors + C(waterfront) + C(view) \
                          + C(condition) + C(grade) + sqft_above + sqft_basement + yr_built + C(renovated) \
                          + C(zipcode) '
    
    smf_model = smf.ols(formula=reg_formula, data=house_df_alt).fit(cov_type='cluster', 
                                                                    cov_kwds={'groups':house_df_alt[cluster_group]})
    print(smf_model.summary())
    return None

In [64]:
reg_model(house_price_alt, 'price', 'zipcode')

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.835
Method:                 Least Squares   F-statistic:                 1.336e+10
Date:                Mon, 17 May 2021   Prob (F-statistic):               0.00
Time:                        16:48:25   Log-Likelihood:            -2.8811e+05
No. Observations:               21613   AIC:                         5.764e+05
Df Residuals:                   21515   BIC:                         5.772e+05
Df Model:                          97                                         
Covariance Type:              cluster                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept            5.596e+05   3